In [1]:
import uuid
import asyncio
import time
from pathlib import Path
import requests

# Pyrit imports
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, HTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.common.text_helper import save_html_report, generate_single_turn_html_report
from pyrit.prompt_target.http_target.parsers.AHAssistantResponseParser import AHAssistantResponseParser

# ------------------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------------------

TOKEN = ""

# Endpoint to create a new thread
create_thread_url = ""

# Endpoint for chat
chat_url = ""

# Headers for all requests
HEADERS = {
    "Content-Type": "application/json",
    "X-Authorization": TOKEN,
    "Accept": "*/*"
}

# Payload for thread creation
thread_init_payload = {
    "message": "Hello, start this chat!",
    "model": "gpt-4o-mini"
}

# HTTP template to send a prompt to a thread
send_prompt_http_template = f"""
    POST {chat_url}
    Content-Type: application/json
    X-Authorization: {TOKEN}
    Accept: */*

    {{
        "message": "{{PROMPT}}"
    }}
"""

# ------------------------------------------------------------------------------
# QA Dataset
# ------------------------------------------------------------------------------

qa_pairs = [
    {"question": "Ik heb zin in fruit", "expected_outcomes": "text_message: Vraag om meer details"},
    {"question": "Appel", "expected_outcomes": "text_message: Apples"}
]

# ------------------------------------------------------------------------------
# Initialize Pyrit Environment
# ------------------------------------------------------------------------------

initialize_pyrit(memory_db_type=IN_MEMORY)

# ------------------------------------------------------------------------------
# Setup Pyrit Targets and Evaluator
# ------------------------------------------------------------------------------

http_prompt_target = HTTPTarget(
    http_request=send_prompt_http_template,
    prompt_regex_string="{PROMPT}",
    timeout=60.0,
    callback_function=AHAssistantResponseParser.parse_response
)

scorer = Evaluator(
    chat_target=OpenAIChatTarget(),
    evaluator_yaml_path=Path("assets/AH_Evaluators/relevance_evaluator.yaml"),
    scorer_type="float_scale"
)

orchestrator = PromptSendingOrchestrator(
    objective_target=http_prompt_target,
    scorers=[scorer]
)

# ------------------------------------------------------------------------------
# Helper: Create Required Number of Chat Threads
# ------------------------------------------------------------------------------

def create_chat_threads(required_count: int, delay_seconds: float = 0.5):
    thread_ids = []
    attempt = 0

    while len(thread_ids) < required_count:
        attempt += 1
        try:
            response = requests.post(create_thread_url, headers=HEADERS, json=thread_init_payload)
            response.raise_for_status()

            thread_id = response.json().get("chatId")
            if thread_id:
                thread_ids.append(thread_id)
            else:
                print(f"[Attempt {attempt}] No chatId in response.")

        except Exception as e:
            print(f"[Attempt {attempt}] Failed to create thread: {e}")
            try:
                print("Raw response:", response.text)
            except:
                pass

        time.sleep(delay_seconds)

    print("✅ All threads created.\n")
    return thread_ids

# ------------------------------------------------------------------------------
# Main Async Function
# ------------------------------------------------------------------------------

async def main():
    questions = [pair["question"] for pair in qa_pairs]
    expected_outcomes = [pair["expected_outcomes"] for pair in qa_pairs]

    start_time = time.time()

    # 1. Create chat threads
    thread_ids = create_chat_threads(required_count=len(qa_pairs))

    # 2. Send prompts into those threads
    await orchestrator.send_prompts_async(
        prompt_list=questions,
        expected_output_list=expected_outcomes,
        thread_ids=thread_ids
    )

    # 3. Collect results and execution time
    results = orchestrator.get_chat_results()
    execution_time = time.time() - start_time

    # 4. Save HTML report
    report_dir = Path("tests/E2E/reports/DataSet").resolve()
    report_dir.mkdir(parents=True, exist_ok=True)

    save_html_report(
        results=results,
        directory=str(report_dir),
        report_generator=generate_single_turn_html_report,
        is_chat_evaluation=False,
        threshold=0.7,
        description="Evaluation of inputs vs. expected/actual outputs with scoring.",
        execution_time=execution_time
    )

# ------------------------------------------------------------------------------
# Run It
# ------------------------------------------------------------------------------

await main()


✅ All threads created.

✅ Report saved at: /Users/denizdalkilic/Documents/Forks/PyRIT/tests/E2E/reports/DataSet/report_20250325_224941.html
